In [ ]:
from model import Att_Gnn
from dataset import AttDataset
from torch_geometric.loader import DataLoader
from loss import weighted_loss
import torch

dataset = AttDataset(root='./data')
data_iter = DataLoader(dataset)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Att_Gnn().to(device)
optimizer = torch.optim.RAdam(model.parameters(), betas=(0.9, 0.999), eps=1.0e-4, lr=1.0e-3, weight_decay=1.0e-4)
loss_func = weighted_loss(dataset[0].edge_index, 0.5)

In [ ]:
model.train()

for epoch in range(10):
    print("epoch: {}".format(epoch))
    for data in data_iter:
        data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = loss_func(out, data)
        loss.requires_grad_(True)
        loss.backward()
        optimizer.step()
        print(loss)

torch.save(model, './models/gnn-model.pkl')

In [3]:
import torch